In [73]:
import chess, glob, tqdm, numpy as np
import chess.pgn
from dataclasses import dataclass

# Preparandos os dados pro modelo

In [75]:
@dataclass
class Jogada:
    estado_tabuleiro: np.ndarray
    origem_jogada: np.ndarray
    destino_jogada: np.ndarray
    peca: np.ndarray
    brancas: int

In [2]:
lista_arquivos_pgn = glob.glob("./download/*.pgn")

In [3]:
def import_pgn_file(file_path: str) -> chess.pgn.Game:
    with open(file_path) as pgn_file:
        game = chess.pgn.read_game(pgn_file)
        return game

In [40]:
def obter_estado_tabuleiro_do_jogo(
    jogo: chess.pgn.Game,
) -> tuple[list[list[int]], dict[int, str]]:
    # Inicializa o tabuleiro
    tabuleiro = jogo.board()

    # Cria uma matriz para representar o tabuleiro
    estado_tabuleiro = [[0] * 8 for _ in range(8)]
    dicionario = {}

    # Preenche a matriz com os valores das peças
    for rank in range(8):
        for arquivo in range(8):
            square = chess.square(arquivo, rank)
            piece = tabuleiro.piece_at(square)
            if piece is not None:
                if piece.color == chess.WHITE:
                    estado_tabuleiro[rank][arquivo] = piece.piece_type
                    if piece.piece_type not in dicionario.keys():
                        dicionario[
                            piece.piece_type
                        ] = f"W_{piece.symbol()}_{piece.unicode_symbol()}"
                else:
                    estado_tabuleiro[rank][arquivo] = piece.piece_type + 6
                    if piece.piece_type + 6 not in dicionario.keys():
                        dicionario[
                            piece.piece_type + 6
                        ] = f"B_{piece.symbol()}_{piece.unicode_symbol()}"

    return estado_tabuleiro, dicionario

In [54]:
def obter_estado_tabuleiro_do_board(
    tabuleiro: chess.Board,
) -> list[list[int]]:
    # Cria uma matriz para representar o tabuleiro
    estado_tabuleiro = [[0] * 8 for _ in range(8)]
    dicionario = {}

    # Preenche a matriz com os valores das peças
    for rank in range(8):
        for arquivo in range(8):
            square = chess.square(arquivo, rank)
            piece = tabuleiro.piece_at(square)
            if piece is not None:
                if piece.color == chess.WHITE:
                    estado_tabuleiro[rank][arquivo] = piece.piece_type
                    if piece.piece_type not in dicionario.keys():
                        dicionario[
                            piece.piece_type
                        ] = f"W_{piece.symbol()}_{piece.unicode_symbol()}"
                else:
                    estado_tabuleiro[rank][arquivo] = piece.piece_type + 6
                    if piece.piece_type + 6 not in dicionario.keys():
                        dicionario[
                            piece.piece_type + 6
                        ] = f"B_{piece.symbol()}_{piece.unicode_symbol()}"

    return estado_tabuleiro

In [41]:
def extrair_movimentos_do_jogo(jogo) -> list[str]:
    movimentos = []
    tabuleiro = jogo.board()

    for move in jogo.mainline_moves():
        movimentos.append(
            move.uci()
        )  # UCI é uma notação padrão para representação de movimentos

    return movimentos

In [66]:
def obter_info_movimento_com_estado(
    move: chess.Move, estado_tabuleiro: list[list[int]]
) -> tuple[list[list[int]], list[list[int]], list[int]]:
    # Obtém as coordenadas da origem e destino do movimento
    origem = [
        [
            1
            if i == chess.square_rank(move.from_square)
            and j == chess.square_file(move.from_square)
            else 0
            for j in range(8)
        ]
        for i in range(8)
    ]
    destino = [
        [
            1
            if i == chess.square_rank(move.to_square)
            and j == chess.square_file(move.to_square)
            else 0
            for j in range(8)
        ]
        for i in range(8)
    ]

    # Calcula a enumeração da peça
    peca_enum = (
        estado_tabuleiro[chess.square_rank(move.from_square)][
            chess.square_file(move.from_square)
        ]
        if estado_tabuleiro[chess.square_rank(move.from_square)][
            chess.square_file(move.from_square)
        ]
        != 0
        else None
    )

    # Cria o vetor de 12 posições com 1 na posição da peça escolhida
    vetor_peca = [1 if i + 1 == peca_enum else 0 for i in range(12)]

    return origem, destino, vetor_peca

In [42]:
arquivo_pgn = import_pgn_file(lista_arquivos_pgn[0])

In [43]:
arquivo_pgn.mainline()

<Mainline at 0x7f11cf182a10 (1. e4 c5 2. Nc3 Nc6 3. g3 g6 4. Bg2 Bg7 5. d3 e6 6. Be3 d6 7. Qd2 Rb8 8. Nge2 Nd4 9. O-O Ne7 10. Nc1 O-O 11. Nd1 b5 12. c3 Ndc6 13. Bh6 b4 14. Bxg7 Kxg7 15. Ne2 d5 16. Qc2 Qa5 17. Ne3 Rd8 18. f4 Qb6 19. Rfb1 Ba6 20. exd5 Nxd5 21. Nxd5 exd5 22. Kh1 Ne7 23. Ng1 Nf5 24. Nf3 Ne3 25. Qd2 d4 26. cxd4 cxd4 27. Re1 Bb7 28. Nh4 Nxg2 29. Nxg2 Re8 30. Kg1 Qc6 31. h3 Rxe1+ 32. Rxe1 Rc8 33. Kh2 a5 34. Nh4 Qc2 35. Re2 Qb1 36. Ng2 Rc1 37. g4 Rh1+ 38. Kg3 Qg1 39. Re4 Qh2+ 40. Kf2 Rg1)>

In [46]:
tabuleiro_atual, dicionario = obter_estado_tabuleiro_do_jogo(arquivo_pgn)

In [47]:
tabuleiro_atual

[[4, 2, 3, 5, 6, 3, 2, 4],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [7, 7, 7, 7, 7, 7, 7, 7],
 [10, 8, 9, 11, 12, 9, 8, 10]]

In [48]:
dicionario

{4: 'W_R_♖',
 2: 'W_N_♘',
 3: 'W_B_♗',
 5: 'W_Q_♕',
 6: 'W_K_♔',
 1: 'W_P_♙',
 7: 'B_p_♟',
 10: 'B_r_♜',
 8: 'B_n_♞',
 9: 'B_b_♝',
 11: 'B_q_♛',
 12: 'B_k_♚'}

In [129]:
# Listando as jogadas
lista_jogadas: list[Jogada] = []

for arquivo in lista_arquivos_pgn:
    arquivo_pgn = import_pgn_file(arquivo)
    tabuleiro = arquivo_pgn.board()

    for movimento in arquivo_pgn.mainline_moves():
        # Estraindo o estado atual do tabuleiro
        tab = obter_estado_tabuleiro_do_board(tabuleiro)

        tabuleiro.push(movimento)
        origem, destino, peca = obter_info_movimento_com_estado(movimento, tab)
        
        # Comprimindo e normalizando o estado do tabuleiro
        tab = np.ravel(np.array(tab, dtype=np.float16))
        tab = tab / 12.0

        lista_jogadas.append(
            Jogada(
                estado_tabuleiro=tab,
                peca=np.array(peca + [0] * ((8**2) - 12 ), dtype=np.float16),
                origem_jogada=np.ravel(np.array(origem, dtype=np.float16)),
                destino_jogada=np.ravel(np.array(destino, dtype=np.float16)),
                brancas=int(tabuleiro.turn == chess.WHITE),
            )
        )

In [130]:
len(lista_jogadas)

19365

In [137]:
X_lista = []
Y_lista = []

for jogada in lista_jogadas:
    x = jogada.estado_tabuleiro
    np.append(x, jogada.brancas)
    y = [jogada.peca, jogada.origem_jogada, jogada.destino_jogada]
    
    
    X_lista.append(x)
    Y_lista.append(y)
    
X = np.array(X_lista, dtype=np.float16)
Y = np.array(Y_lista, dtype=np.float16)

# Criando o modelo

In [138]:
from keras import Sequential
from keras.layers import LSTM, Input, Dense, Dropout
import keras

In [139]:
camada_saida_peca = Dense(12, activation="linear", name="saida_peca")
camada_saida_origem = Dense(8**2, activation="linear", name="saida_origem")
camadaa_saida_destino = Dense(8**2, activation="linear", name="saida_destino")

In [146]:
modelo = Sequential(
    [
        LSTM(units=32, return_sequences=True, input_shape=(X.shape[1], 1)),
        Dropout(0.3),
        LSTM(units=8**2, return_sequences=True),
        Dropout(0.3),
        LSTM(units=8**2, return_sequences=True),
        Dropout(0.3),
        LSTM(units=8**2, return_sequences=True),
        Dropout(0.3),
        Dense(64, activation="linear", name="saida_peca"),
        Dense(64, activation="linear", name="saida_origem"),
        Dense(64, activation="linear", name="saida_destino"),
    ]
)

In [147]:
modelo.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [148]:
historico = modelo.fit(X, Y, epochs=50, batch_size=64)

Epoch 1/50


ValueError: in user code:

    File "/home/gus/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/gus/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/gus/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/home/gus/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/gus/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1185, in compute_loss
        return self.compiled_loss(
    File "/home/gus/.local/lib/python3.10/site-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_

In [145]:
Y.shape

(19365, 3, 64)

In [128]:
Y[0]

[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float16),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float16),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float16)]